# 0.Name & Z-ID (5 pts)
Mounika Thudi \
Z1948596 \
Assignment 6

# 1. Product Classes (20 pts)

In [1]:
#StoreItem is the baseclass
class StoreItem:
    name = ""
    sku = ""
    category = ""
    price = ""
    
    def __init__(self, name, sku,price):
        self.name = name
        self.sku = sku
        self.price = price
    def __str__(self):
        return f"{self.name} ({self.sku}) @ {self.price}"
    
#GroceryItem subclasses from  StoreItem      
class GroceryItem(StoreItem):
    def __init__(self, name,sku,price,nutrition={}):
        super().__init__(name,sku,price)
        self.nutrition = nutrition    
    def __str__(self):
        return f"{self.name} ({self.sku}) @ {self.price}"
    
#HouseholdItem subclasses from  StoreItem
class HouseholdItem (StoreItem):
    brand = ""
    def __init__(self, name, sku,price,brand):
        super().__init__(name,sku,price)
        self.brand = brand
    def __str__(self):
        return f"{self.name} ({self.sku}) @ {self.price} {self.brand}"
    
#BulkItem subclasses from  StoreItem
class BulkItem(GroceryItem):
    nutrition={}
    unit = ""
    def __init__(self, name, sku,price,unit):
        super().__init__(name,sku,price)
        self.unit = unit 
    def __str__(self):
        return f"{self.name} ({self.sku}) @ {self.price}/{self.unit}"
        
cereal = GroceryItem("Cheerios", 38942, 3.99, {'calories': 200, 'fat': 0})
bananas = BulkItem("Bananas", 4011, 0.59,"lb")
print(f"Two items: {cereal}, {bananas}")


Two items: Cheerios (38942) @ 3.99, Bananas (4011) @ 0.59/lb


# 2. Inventory Class (20 pts)

In [2]:
class inventory:
    
    def __init__(self):
        self.inventory_store={}
        
    # add_items method    
    def add_items(self,StoreItem_object,amount):
        self.inventory_store[StoreItem_object.sku]={'item':StoreItem_object,'amount':amount}
        
    # remove_items method
    def remove_items(self,StoreItem_object,amount):
        if self.has_enough(StoreItem_object,amount):
            self.inventory_store[StoreItem_object['item'].sku]['amount']-= amount
        else:
            raise Exception ("Inventory doesn't exist these many items in stock, there are less items than you need")
            
    # find_item method by sku,if sku doesn't exist raise keyerror
    def find_item(self,sku_id):
        for sku,dict in self.inventory_store.items():
            if sku_id ==sku:
                return self.inventory_store[sku_id]
            
            
        raise KeyError("No item with particular skuid,Item searched is not present in the inventory stock, search for other items you need")

    
    # has_enough method to check for number for items required are in stock or not            
    def has_enough(self,StoreItem_object,amount):
        for x,y in self.inventory_store.items():
            if (id(y['item'])==id(StoreItem_object)):          # check for item
                return True if y['amount']>=amount else False  # check for amount
                    
    # method for human readable string            
    def __str__(self):
        result = ""
        for obj in self.inventory_store.values():
            result+= f"{obj['amount']} {obj['item']}\n"
        return result

# 3. ShoppingCart Class (20 pts) and Added Extra Credit- Transfer shopping cart items

In [3]:
class ShoppingCart: 
    def __init__(self,inventory):
        self.cart_store={}
        self.inventory = inventory
        
    def add_items(self,StoreItem_object,amount):
        if (not self.inventory.has_enough(StoreItem_object['item'],amount)):
            raise Exception ("Inventory doesn't exist these many items in stock, there are less items than you need")
        if (not float(amount).is_integer()) and not isinstance (StoreItem_object['item'],BulkItem):
            raise Exception("Only Bulk items do allow fractional amounts!, Other Items doesn't allow non-integer amount of an item")
        else:
            self.cart_store[StoreItem_object['item'].sku] = {'item':StoreItem_object['item'],'amount':amount}
             
        
    def remove_items(self,StoreItem_object,amount):
        try:
            self.cart_store[StoreItem_object['item'].sku] 
        except Exception:
            print("Item searched to remove from shopping cart doesn't exist in shopping cart")
        else:
            self.cart_store[StoreItem_object['item'].sku]['amount']-=amount
    
    def __add__(self,StoreItem_object):
        self.cart[StoreItem_object.sku]['amount']+=1
        
    def __sub__(self,StoreItem_object):
        self.cart_store[StoreItem_object.sku]['amount']-=1
                
    def cost(self):
        total_cost=0
        for obj in self.cart_store.values():
            total_cost+= obj['item'].price*int(obj['amount'])
        return total_cost
    
    # Extra credits to transfer items from one shopping cart to other
    def transfer_items(self, cart):
        for obj in cart.cart_store.values():
            self.cart_store[obj['item'].sku] = {'item':obj['item'],'amount': obj['amount']}

    def __str__(self):
        result = ""
        for obj in self.cart_store.values():
            result+= f"{obj['amount']} {obj['item']}\n"
        return result

# 4. Store Class (20 pts) and Extra Credit support in other way

In [4]:
class Store:
    MAX_CARTS=10
    def __init__(self):
        self.shopping_carts_data ={}
        self.carts_number= 0
        self.inventory=inventory()
    
    def  get_cart(self):
        self.carts_number+=1
        if self.carts_number > Store.MAX_CARTS:
            raise Exception('Too many carts are in use, MAX carts in store are 10 only')
        
        new_shopping_cart = ShoppingCart(self.inventory)
        self.shopping_carts_data[id(new_shopping_cart)]=new_shopping_cart
        return new_shopping_cart
       
            
    def checkout(self,new_shopping_cart):
        cart_items= self.shopping_carts_data[id(new_shopping_cart)]
        for obj in cart_items.cart_store.values():
            #print(self.shopping_carts_data[id(cart)])
            sku_id=(obj['item'].sku)
            self.inventory.find_item(sku_id)['amount']-=obj['amount'] # removes items added to shopping cart from the inventory
        total_amount = cart_items.cost() # calculates total cost of shopping cart
        
        del cart_items   #removecart from the store
        return total_amount
    
    # Extra credit support for transferring items from one shopping cart to another ( tried in another way)
    def shopping_cart_transfer_items(self,source_cart,destination_cart,StoreItem_object):
        #source cart items are stored by taking id of source_cart
        self.shopping_carts_data[id(source_cart)]  = source_cart
        source_cart_items = self.shopping_carts_data[id(source_cart)] 
        #in this line of code transfer item object is taken
        transfer_unit = source_cart_items.cart_store[StoreItem_object['item'].sku] 
        #initial destination_cart items  by taking id of destination_cart
        self.shopping_carts_data[id(destination_cart)] = destination_cart
        destination_cart_items= self.shopping_carts_data[id(destination_cart)]
        # adding items to the destination cart
        destination_cart_items.add_items(transfer_unit,transfer_unit['amount'])
        #deleting transfered unit from the source_cart
        del source_cart_items.cart_store[StoreItem_object['item'].sku]
        #returning souce_cart_items and destination_cart_items
        return source_cart_items,destination_cart_items
                    

# 6. Summary and Testing

In [12]:
store = Store()
store.inventory.add_items(GroceryItem("Cheerios", 38942, 3.99, {'calories': 200, 'fat': 0}), 10)
store.inventory.add_items(HouseholdItem("Trash Bags", 38902, 5.99, "Hefty"), 30)
store.inventory.add_items(BulkItem("Bananas", 4011, 0.59, "lb"), 20)
store.inventory.add_items(BulkItem("Lettuce", 3982, 2.99, "head"), 25)
store.inventory.add_items(GroceryItem("Oreos", 27894, 3.29), 1)

print("Inventory:")
print(store.inventory)
my_cart = store.get_cart()
my_cart.add_items(store.inventory.find_item(4011), 3)
my_cart.add_items(store.inventory.find_item(38902),1)
my_cart.add_items(store.inventory.find_item(27894), 1)
print("Cart:")
print(my_cart)

total_cost = store.checkout(my_cart)
print(f"You spent ${total_cost}")

print("Updated Inventory:")
print(store.inventory)

Inventory:
10 Cheerios (38942) @ 3.99
30 Trash Bags (38902) @ 5.99 Hefty
20 Bananas (4011) @ 0.59/lb
25 Lettuce (3982) @ 2.99/head
1 Oreos (27894) @ 3.29

Cart:
3 Bananas (4011) @ 0.59/lb
1 Trash Bags (38902) @ 5.99 Hefty
1 Oreos (27894) @ 3.29

You spent $11.05
Updated Inventory:
10 Cheerios (38942) @ 3.99
29 Trash Bags (38902) @ 5.99 Hefty
17 Bananas (4011) @ 0.59/lb
25 Lettuce (3982) @ 2.99/head
0 Oreos (27894) @ 3.29



# Extra credit- support for transfering items from one cart to another

In [13]:
new_cart = store.get_cart()
new_cart.add_items(store.inventory.find_item(3982),1)
print(new_cart)

1 Lettuce (3982) @ 2.99/head



In [14]:
my_cart = store.get_cart()
my_cart.add_items(store.inventory.find_item(4011), 3)
my_cart.add_items(store.inventory.find_item(38902),1)
print("Before transfer cart items:")
print(my_cart)
my_cart.transfer_items(new_cart)
print("After transfer items:")
print(my_cart)

Before transfer cart items:
3 Bananas (4011) @ 0.59/lb
1 Trash Bags (38902) @ 5.99 Hefty

After transfer items:
3 Bananas (4011) @ 0.59/lb
1 Trash Bags (38902) @ 5.99 Hefty
1 Lettuce (3982) @ 2.99/head



# 5. [CS503 Only] Exceptions (15 pts)

In [15]:
my_cart = store.get_cart()
my_cart.add_items(store.inventory.find_item(4013), 3) # raises your  exception

KeyError: 'No item with particular skuid,Item searched is not present in the inventory stock, search for other items you need'

In [9]:
my_cart.add_items(store.inventory.find_item(38902), 1.5) # raises your exception

Exception: Only Bulk items do allow fractional amounts!, Other Items doesn't allow non-integer amount of an item

In [10]:
store.inventory.remove_items(store.inventory.find_item(27894), 3) # raises your exception

Exception: Inventory doesn't exist these many items in stock, there are less items than you need

In [11]:
for i in range(20):  # too many carts exception
    store.get_cart()

Exception: Too many carts are in use, MAX carts in store are 10 only